In [ ]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [ ]:
def createsentances(data):
    games = data["games"]
    card_history = []
    for i in range(len(games)):
        g = games[i]
        if not(g.has_key("card_history")) or g["card_history"] != []:
            card_history.append(pd.DataFrame(g["card_history"]))
    sentances = []
    for i in range(len(card_history)):
        c = card_history[i]
        me = c[c.player=="me"]
        op = c[c.player=="opponent"]
        me = me["card"]
        op = op['card']
        sentances.append([i["name"] for i in me])
        sentances.append([i["name"] for i in op])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [ ]:
def createsentancesfiles(listfile):
    sentances = []
    for e in listfile:
        data=pd.read_json(listfile)
        sentances += createsentances(data)
    return sentances

In [ ]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


sentances = createsentancesfiles(filesHearthstoneGames)

In [ ]:
data = pd.read_json("2018-01.zip");

In [ ]:
sentances = createsentances(data)

In [ ]:
sentances = []

In [ ]:
for e in filesHearthstoneGames:
    data = pd.read_json(e);
    sentances +=createsentances(data)
    


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [ ]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHeartStone = requests.get(urlApi).json()

In [ ]:
listNameCard = [e["name"] for e in ApiHeartStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [ ]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [1]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de layers
workers paralléliser sur un nombre de coeurs

In [ ]:
model = Word2Vec(sentancesFiltered,min_count=10,size=200,workers=4)

sauvegarde du model

In [ ]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [3]:
model = Word2Vec.load("word2vecModel")

test de la  similarité

In [4]:
model.wv.similarity("Open the Waygate","Fireball")

0.48961156126067124

In [10]:
model.wv.most_similar(positive=["Open the Waygate","Fireball","The Lich King"])

[(u'Primordial Glyph', 0.8065460920333862),
 (u'Ice Block', 0.8050417304039001),
 (u'Shatter', 0.8047003746032715),
 (u'Blizzard', 0.7947472333908081),
 (u'Arcane Intellect', 0.7853944897651672),
 (u'Servant of Yogg-Saron', 0.784844160079956),
 (u'Freezing Potion', 0.7802477478981018),
 (u'Babbling Book', 0.7764003872871399),
 (u'Cone of Cold', 0.7715980410575867),
 (u'Arcane Artificer', 0.7667562961578369)]